<a href="https://colab.research.google.com/github/iannjari/scrapbook/blob/main/Clean_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dependencies for data import

Un-comment the code cell below to obtain the data. Blocks should be seperated to test whether they're individually working.

In [1]:
'''
import requests
import tarfile
from os import path

# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()

# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

import pandas as pd 
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape

'''

'\nimport requests\nimport tarfile\nfrom os import path\n\n# Downloading the dataset\n\nfname = \'airline_2m.tar.gz\'\nurl = \'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/\' + fname\nr = requests.get(url)\nopen(fname , \'wb\').write(r.content)\n\n# Extracting the dataset\ntar = tarfile.open(fname)\ntar.extractall()\ntar.close()\n\n# Verifying the file was extracted properly\ndata_path = "airline_2m.csv"\npath.exists(data_path)\n\nimport pandas as pd \ndf = pd.read_csv(data_path, encoding = "ISO-8859-1",\n                 dtype={\'Div1Airport\': str, \'Div1TailNum\': str, \'Div2Airport\': str, \'Div2TailNum\': str})\ndf.shape\n\n'

Getting the features (these were obtained by reviewing the data by datatype, description from the US DoT(helped remove redudant features) on excel, removing unwanted columns, re-reading the data and converting them to the dictionary below)

In [2]:
import pandas as pd

'''path = "airline_2m.csv"
df = pd.read_csv(path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape'''

'path = "airline_2m.csv"\ndf = pd.read_csv(path, encoding = "ISO-8859-1",\n                 dtype={\'Div1Airport\': str, \'Div1TailNum\': str, \'Div2Airport\': str, \'Div2TailNum\': str})\ndf.shape'

In [3]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DOT_ID_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginState',
 'DestAirportID',
 'DestState',
 'DepTime',
 'Cancelled']

In [4]:
#commmented since featuredata CSV is available featuredata=df[featurecols]
'''
featuredata=featuredata.dropna()
featuredata.to_csv('featuredata.csv',index=False)
'''

"\nfeaturedata=featuredata.dropna()\nfeaturedata.to_csv('featuredata.csv',index=False)\n"

In [5]:
featuredata=pd.read_csv('featuredata.csv')
featuredata=featuredata.dropna()
featuredata

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,Cancelled
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912545,1993,2,6,19,6,19805,799,14843,PR,13303,FL,729.0,0.0
1912546,1990,3,8,7,2,19790,379,14107,AZ,12889,NV,1010.0,0.0
1912547,1999,4,11,15,1,19805,1670,14843,PR,11618,NJ,1217.0,0.0
1912548,2019,1,3,5,2,19393,1944,14831,CA,12892,CA,708.0,0.0


In [6]:
target_name = "Cancelled"
target = featuredata[target_name]
# note that all features are categorical in nature
data = featuredata.drop(columns=[target_name])

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    data,target, test_size=0.33, random_state=42)


In [8]:
target.value_counts()

0.0    1912101
1.0        449
Name: Cancelled, dtype: int64

Class imbalance might be an issue with models modelled on this data

In [9]:
data.isnull().values.any()

False

Use a dummy classifier that predicts the majority class always as the base model

In [10]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_train, y_train)


dummy_clf.predict(x_test)
dummy_clf.score(x_test, y_test)

0.9997639200053237

High accuracy,arising from the imbalanced classes in the target.

What's the performance of a logistic regression?

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, x_train, y_train, cv=10,return_estimator=True)

cv_results


{'estimator': [Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticReg

In [12]:
array=cv_results["test_score"]
array.mean()

0.9997658825292154

Cross-validation score of 99.97 percent

How will it perform on unseen data?

In [13]:
logistic_regression.fit(x_train, y_train)
predictions=logistic_regression.predict(x_test)
logistic_regression.score(x_test,y_test)

0.9997639200053237

Look at other metrics

In [14]:
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(y_test, predictions)
tn, fp, fn, tp = c_matrix.ravel()

precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= nan
Misclassification= 0.00023607999467631689
F1 score= 0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


In [15]:
tn,fp,fn,tp

(630993, 0, 149, 0)

In [16]:
y_test.value_counts()

0.0    630993
1.0       149
Name: Cancelled, dtype: int64

Model learnt to predict most common class due to class imbalance

In [17]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target 

x_ros, y_ros = ros.fit_resample(x_train, y_train)

y_ros=pd.DataFrame(y_ros)

print('Original dataset shape', y_train.value_counts())
print('Resample dataset shape', y_ros.value_counts())

Original dataset shape 0.0    1281108
1.0        300
Name: Cancelled, dtype: int64
Resample dataset shape Cancelled
1.0          1281108
0.0          1281108
dtype: int64


In [19]:
y_ros

,Cancelled
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
2562211,1.0
2562212,1.0
2562213,1.0
2562214,1.0


In [21]:
logistic_regression2 = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression(max_iter=1000)
)


cv_results = cross_validate(
    logistic_regression2, x_ros, y_ros['Cancelled'], cv=10,return_estimator=True)

cv_results

{'estimator': [Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression(max_iter=1000))]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression(max_iter=1000))]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression(max_iter=1000))]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression(max_iter=1000))]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression(max_iter=1000))]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression(max_iter=1000))]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(h

In [22]:
array=cv_results["test_score"]
array.mean()

0.9980161704524532

In [24]:
y_ros=y_ros['Cancelled']

Cross-validation score of 99.79 percent

How will it perform on unseen data?

In [25]:
logistic_regression2.fit(x_ros,y_ros)
predictions=logistic_regression2.predict(x_test)
logistic_regression2.score(x_test,y_test)

0.9962781751174854

In [26]:
c_matrix=confusion_matrix(y_test, predictions)
tn, fp, fn, tp = c_matrix.ravel()
precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= 0.0031616982836495033
Misclassification= 0.003721824882514553
F1 score= 0.005924672027084215


In [27]:
tp,tn,fp,fn

(7, 628786, 2207, 142)

Try undersampling

In [28]:
# Training will be done on smaller dataset

# Create dataframe from oversampled data
df=pd.DataFrame(x_ros)
df['target']=y_ros
df

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,target
0,2015,3,7,5,7,20398,3275,13930,IL,10693,TN,1956.0,0.0
1,2001,2,5,5,6,20355,408,10135,PA,14122,PA,1540.0,0.0
2,1988,1,3,19,6,19790,458,12451,FL,10397,GA,1845.0,0.0
3,2000,2,4,22,6,19805,1621,12953,NY,15304,FL,1545.0,0.0
4,2009,4,11,17,2,19393,258,14057,OR,11884,WA,1733.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562211,2014,1,1,6,1,20366,5037,10980,TN,10397,GA,713.0,1.0
2562212,2019,3,7,22,1,19690,3,12892,CA,12173,HI,1013.0,1.0
2562213,2012,2,5,4,5,20366,5935,11292,CO,10620,MT,1437.0,1.0
2562214,2010,4,12,12,7,20363,3829,11433,MI,11003,IA,1909.0,1.0


In [29]:
# Sample data
df2=df.sample(n=30000)
df2['target'].value_counts()

0.0    15031
1.0    14969
Name: target, dtype: int64

In [30]:
x,y = df2.drop(columns=['target']),df2['target']
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y,random_state=42)

x_test=x_test.to_numpy()
x_train=x_train.to_numpy()

Try stacked classifiers

In [31]:
from sklearn.ensemble import (RandomForestClassifier, StackingClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [32]:


clf=StackingClassifier(estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression())

In [33]:
pipeline = make_pipeline(OneHotEncoder(handle_unknown = 'ignore'),
            StackingClassifier(estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression()))

pipeline.fit(x_train,y_train)
pipeline.score(x_test,y_test)

0.9998666666666667

In [34]:
preds=pipeline.predict(x_test)
c_matrix=confusion_matrix(y_test, preds)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(3757, 1, 0, 3742)

In [35]:
precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= 0.9997328346246327
Misclassification= 0.00013333333333333334
F1 score= 0.9998663994655979


Save this model in ONNX format

In [36]:
pip install skl2onnx

     |████████████████████████████████| 265 kB 13.7 MB/s 
     |████████████████████████████████| 12.7 MB 62 kB/s 
     |████████████████████████████████| 77 kB 4.5 MB/s 


In [38]:
import skl2onnx 
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType




initial_types =  [('feature_input', FloatTensorType([None, 8])),
                  ('feature_input', StringTensorType([None,1])),
                  ('feature_input', FloatTensorType([None, 1])),
                  ('feature_input', StringTensorType([None,1])),
                  ('feature_input', FloatTensorType([None, 1]))]

onx = convert_sklearn(logistic_regression,initial_types=initial_types)

with open("stacked_clf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(x_train)
embeddings = vectorizer.transform(x_train)
embeddings = embeddings.todense()
embeddings
#dim = embeddings.shape[1]
#dim

In [44]:
enc=OneHotEncoder(handle_unknown = 'ignore')
matrix=enc.fit_transform(x_train)
df=pd.DataFrame.sparse.from_spmatrix(matrix)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6221,6222,6223,6224,6225,6226,6227,6228,6229,6230,6231,6232,6233,6234,6235,6236,6237,6238,6239,6240,6241,6242,6243,6244,6245,6246,6247,6248,6249,6250,6251,6252,6253,6254,6255,6256,6257,6258,6259,6260
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
pipeline = make_pipeline(OneHotEncoder(handle_unknown = 'ignore',sparse=False),
            StackingClassifier(estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression()))

pipeline.fit(x_train,y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('stackingclassifier',
                 StackingClassifier(estimators=[('rf',
                                                 RandomForestClassifier(n_estimators=10,
                                                                        random_state=42)),
                                                ('gb',
                                                 GradientBoostingClassifier(n_estimators=10,
                                                                            random_state=42)),
                                                ('knn',
                                                 KNeighborsClassifier())],
                                    final_estimator=LogisticRegression()))])

In [54]:
pipeline.score(x_test,y_test)

0.9998666666666667

In [62]:
initial_types =  [('feature_input', FloatTensorType([None, 8])),
                  ('feature_input', StringTensorType([None,1])),
                  ('feature_input', FloatTensorType([None, 1])),
                  ('feature_input', StringTensorType([None,1])),
                  ('feature_input', FloatTensorType([None, 1]))]

#onx = to_onnx(pipeline,initial_types=initial_types)
onx=skl2onnx.to_onnx(pipeline, X=x_test[0])

with open("stacked_clf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


RuntimeError: ignored

In [60]:
x_test

array([[2007, 2, 5, ..., 12896, 'TX', 2112.0],
       [2017, 3, 8, ..., 13487, 'MN', 1419.0],
       [1993, 1, 2, ..., 12892, 'CA', 1859.0],
       ...,
       [2008, 3, 8, ..., 12173, 'HI', 1127.0],
       [2012, 1, 1, ..., 12441, 'WY', 1125.0],
       [1998, 2, 4, ..., 13198, 'MO', 1941.0]], dtype=object)